# Cleaning Data

Once the .csv files are in the right folders, run this notebook to clean the datasets for embedding generation

The aim of this noteook is to make sure all the sales and basket products have product information associated with them

Imports

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats

from sklearn.impute import SimpleImputer as Imputer  # Specific function from common machine learning package

import glob
import os
from pathlib import Path

Paths

In [ ]:
product_standard_path = "Data/product/product_standard_attributes.csv"
product_advanced_path = "Data/product/product_detailed_attributes.csv"
store_path = "Data/store/store_attributes.csv"
sales_path = "Data/weekly_sales"
basket_path = "Data/basket_data"

Clean Standard Products

In [ ]:
df_product_standard = pd.read_csv(product_standard_path)

In [ ]:
null_product_index = df_product_standard.loc[df_product_standard.isnull().sum(1)>10].index
df_product_standard.drop(null_product_index, axis=0, inplace=True)

df_product_standard.fillna({'package_depth_qty':0, 'package_height_qty':0, 'package_width_qty': 0,'package_volume_qty':0,'package_weight_qty':0,'national_consumer_price_amt':0 }, inplace=True)
df_product_standard.fillna('',inplace=True)
df_product_standard = df_product_standard[df_product_standard["ctr_product_num"] != "STORE_PRODUCT"] 
df_product_standard["ctr_product_num"] = df_product_standard["ctr_product_num"].astype(np.int64)
df_product_standard["product_creation_date"] = df_product_standard["product_creation_date"].astype('datetime64[ns]')
df_product_standard["corporate_status_cd_change_date"] = df_product_standard["corporate_status_cd_change_date"].astype('datetime64[ns]')

In [ ]:
try:
    os.mkdir("clean_data")
except:
    None

In [ ]:
df_product_standard.to_csv("clean_data\\cleaned_products.csv")

In [ ]:
#df_product_standard = None
null_product_index = None

Clean Product Detail

In [ ]:
df_product_detailed = pd.read_csv(product_advanced_path)

In [ ]:
df_product_detailed

In [ ]:
df_product_detailed["product_num"] = df_product_detailed["product_num"].astype('Int64')
df_product_detailed.dtypes
df_product_detailed = df_product_detailed.drop(df_product_detailed[df_product_detailed.product_num.isna()].index)
df_product_detailed = df_product_detailed.rename(columns={'product_num': 'ctr_product_num'})
df_product_detailed = df_product_detailed[~df_product_detailed.attr_value_en_txt.isin(['Features and Benefits not loaded','NaN','Features and benefits not loaded','Features and Benefits not loaded','Features and Benefits not loaded,','Na'])]
df_product_detailed.attr_lov_value_id = df_product_detailed.attr_lov_value_id.fillna('')
df_product_detailed.attr_value_en_txt = df_product_detailed.attr_value_en_txt.fillna('')
df_product_detailed[["attribute_id","attr_lov_value_id", "attr_value_en_txt"]] = df_product_detailed[["attribute_id","attr_lov_value_id", "attr_value_en_txt"]].astype(str)
df_product_detailed.groupby("ctr_product_num").head()
df_product_detailed['attr_value_en_sentence'] = df_product_detailed.groupby('ctr_product_num')['attr_value_en_txt'].transform(lambda x: ''.join(x))
df_product_detailed['attr_value_en_sentence'] = df_product_detailed['attr_value_en_sentence'].str.strip(',')
df_product_detailed['attr_value_en_sentence'] = df_product_detailed['attr_value_en_sentence'].str.strip(' ')
df_product_detailed['attr_value_en_sentence'] = df_product_detailed['attr_value_en_sentence'].str.strip('.')

In [ ]:
df_product_detailed

In [ ]:
df_product_detailed.to_csv("clean_data\\cleaned_products_detailed.csv")

In [ ]:
df_product_detailed = None

Clean Stores

In [ ]:
df_store = pd.read_csv(store_path)

In [ ]:
df_store = df_store.drop(columns=['soft_opening_date', 'grand_opening_date'])
df_store = df_store.dropna(subset=['store_nm','province_cd','latitude_qty','longitude_qty','store_size_cd','retail_square_ft_qty','checkouts_count','store_concept_type_nm','winterized_canopy_txt'])

values = {"ins_garden_centre_sqr_ft_qty": 0, "number_of_service_bays_qty": 0, "onsite_propane_txt": 'No', "shopping_centre_nm": '0'}
df_store = df_store.fillna(value=values)

# drop fake store
df_store=df_store[~df_store['store_num'].isin([885,949])]

change_type_list = ['retail_square_ft_qty','ins_garden_centre_sqr_ft_qty','number_of_service_bays_qty','checkouts_count']
df_store[change_type_list] = df_store[change_type_list].astype(int)

In [ ]:
df_store.to_csv("clean_data\\cleaned_store.csv")

Clean Sales

In [ ]:
all_files = Path(sales_path).glob('*.csv') 

dfs = []
for file in all_files:
    dfs.append(pd.read_csv(file))
    
df_sales_data = pd.concat(dfs, ignore_index=True)
dfs = None # Clear memory

In [ ]:
df_sales_data = df_sales_data[df_sales_data.product_num != "STORE_PRODUCT"]
df_sales_data["product_num"] = df_sales_data["product_num"].astype(np.int64)
df_sales_data = df_sales_data[df_sales_data["product_num"].isin(df_product_standard["ctr_product_num"].unique())
                                 & df_sales_data["store_num"].isin(df_store["store_num"].unique())]

In [ ]:
df_sales_data.to_csv("clean_data\\cleaned_sales.csv")
df_sales_data = None

Clean Basket

In [ ]:
all_files = Path(basket_path).glob('*.csv') 

dfs = []
for file in all_files:
    dfs.append(pd.read_csv(file))
df_basket_data = pd.concat(dfs, ignore_index=True)
dfs = None


In [ ]:
df_basket_data = df_basket_data[df_basket_data["product_num"] != "STORE_PRODUCT"]
df_basket_data["product_num"] = df_basket_data["product_num"].astype(np.int64)
df_basket_data["transaction_date"] = df_basket_data["transaction_date"].astype('datetime64[ns]')

df_basket_data = df_basket_data[df_basket_data["product_num"].isin(df_product_standard["ctr_product_num"].unique())
                                 & df_basket_data["store_num"].isin(df_store["store_num"].unique())]

In [ ]:
df_basket_data.to_csv("clean_data\\cleaned_basket.csv")
df_basket_data = None